In [1]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertConfig
from datasets import load_dataset,load_metric
import numpy as np


from accelerate import Accelerator


accelerator = Accelerator()
device = accelerator.device

/opt/crc/c/conda/23.5.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [30]:
# texts = "This movie was the best movie I have ever seen! some scenes were ridiculous, but acting was great."
texts = "I really didn't like this movie. Some of the actors were good, but overall the movie was boring."
# texts = "I hate that I love you."

pretrained_name = "distilbert-base-uncased-finetuned-sst-2-english"

pred_config = DistilBertConfig.from_pretrained(pretrained_name)
pred_tokenizer = DistilBertTokenizer.from_pretrained(pretrained_name)
pred_model = DistilBertForSequenceClassification.from_pretrained(pretrained_name).to(device)

inputs = pred_tokenizer(texts, return_tensors="pt")
with torch.no_grad():
    inputs = {key:val.to(device) for key,val in inputs.items()}
    logits = pred_model(**inputs).logits

predicted_class_id = logits.argmax().item()
print(pred_model.config.id2label[predicted_class_id])

print(inputs)

NEGATIVE
{'input_ids': tensor([[  101,  1045,  2428,  2134,  1005,  1056,  2066,  2023,  3185,  1012,
          2070,  1997,  1996,  5889,  2020,  2204,  1010,  2021,  3452,  1996,
          3185,  2001, 11771,  1012,   102]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]], device='cuda:0')}


In [31]:
from maskgen.text_models.text_maskgen_model import MaskGeneratingModel
pred_hidden_dim = pred_model.config.dim
num_labels = pred_model.config.num_labels

mask_gen_model = MaskGeneratingModel(pred_model, hidden_size=pred_hidden_dim, num_classes=num_labels)
mask_gen_model.to(device)

mask_gen_model.load_state_dict(torch.load('text_mask_gen_model/mask_gen_model_0_30.pth'))
# mask_gen_model.load_state_dict(torch.load('trained/mask_gen_model12/mask_gen_model_2_90.pth'))
mask_gen_model.eval()
print()

In [32]:
from captum.attr import visualization
expl = mask_gen_model.attribute_text(inputs['input_ids'], inputs['attention_mask'])[0]
print(expl)
tokens = pred_tokenizer.convert_ids_to_tokens(inputs['input_ids'].flatten())
# normalize expl
expl = (expl - expl.min()) / (expl.max() - expl.min())
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                0,
                                0,
                                0,
                                0,
                                1,       
                                tokens,
                                1)]
                            
visualization.visualize_text(vis_data_records)

tensor([0.3663, 0.3980, 0.4280, 0.4310, 0.4675, 0.4304, 0.4485, 0.4172, 0.4135,
        0.4655, 0.4482, 0.4663, 0.4502, 0.4503, 0.4511, 0.4646, 0.4732, 0.3917,
        0.3688, 0.3774, 0.3830, 0.3688, 0.3738, 0.4421, 0.3932],
       device='cuda:0')


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
0,0 (0.00),0,1.00,"[CLS] i really didn ' t like this movie . some of the actors were good , but overall the movie was boring . [SEP]"


In [33]:
expl.shape

torch.Size([25])

In [16]:
len(tokens)

21